#### Writing from DataFrame to Lakehouse Table

In [1]:
# first, let's get some data 
df = spark.read.json('Files/pyspark/json/property-sales.json')

display(df)

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, bfde1227-6350-40de-84c1-c0acf144b7d2)

#### Beware of the limitations of Lakehouse column naming
- Read more [here](https://learn.microsoft.com/en-us/fabric/data-engineering/load-to-tables) 

In [2]:
#inspecting the schema 
df.printSchema()

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 4, Finished, Available)

root
 |-- Address : string (nullable = true)
 |-- Agent: string (nullable = true)
 |-- City : string (nullable = true)
 |-- SalePrice ($): string (nullable = true)
 |-- Type: string (nullable = true)



In [3]:
# changing column names to allow write to Lakehouse tables
df = df.withColumnRenamed("SalePrice ($)","SalePrice_USD")\
        .withColumnRenamed("Address ", "Address")\
        .withColumnRenamed("City ", "City")
display(df)

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4104b977-53b2-4719-b469-dbb11e903d76)

In [4]:
df.printSchema()

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 6, Finished, Available)

root
 |-- Address: string (nullable = true)
 |-- Agent: string (nullable = true)
 |-- City: string (nullable = true)
 |-- SalePrice_USD: string (nullable = true)
 |-- Type: string (nullable = true)



#### Writing DF to Table, with different 'modes'
Using saveAsTable, we save the DataFrame as a 'Managed Table' (Spark terminology) - meaning both the metadata and the data is managed by Spark.

With a managed table, because Spark manages everything, a SQL command such as DROP TABLE table_name deletes both the metadata and the data. With an unmanaged table, the same command will delete only the metadata, not the actual data.

In [5]:
delta_table_name = 'PropertySales'

# use saveAsTable to save as a Managed Table
df.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)


StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 7, Finished, Available)

**Four different write modes**

In [6]:
# these are four different write 'modes' 

# append the new dataframe to the existing Table
df.write.mode("append").format("delta").saveAsTable(delta_table_name)

display(df)

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, 67ebab3a-5402-4df6-b039-660bc2890b75)

In [7]:
# overwrite existing Table with new DataFrame
df.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)

display(df)

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3ce6c413-21a8-47f2-9328-6a5cd842d4ce)

In [8]:
# Throw error if data already exists
df.write.mode("error").format("delta").saveAsTable(delta_table_name)

display(df)

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 58a94a79-480c-4f50-be09-fb781a286952)

In [9]:
# Fail silently if data already exists 
df.write.mode("ignore").format("delta").saveAsTable(delta_table_name)

display(df)

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 33a8ac12-a5ee-454a-8d15-5fd4ab1ee323)

#### Write to an unmanaged delta table (perhaps for export to external file system/ Databricks/ Snowflake)

In [10]:
# unmanaged table
df.write.mode("overwrite").format("delta").save(path="Files/pyspark/delta/unmanaged.delta")


StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 12, Finished, Available)

#### Read from Table into DataFrame

In [11]:
df = spark.sql("SELECT * FROM OneLake_Shaun.propertysales LIMIT 1000")
display(df)

StatementMeta(, 993a1c49-58b3-410a-bf0f-6bf241fa0889, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 571e8211-32d2-47d0-8c8a-ecb1b2945bd6)